In [1]:
import pandas as pd
import os 
import numpy as np

fpath = os.path.join(os.getcwd(),"data")

fpower = os.path.join(fpath, "230420 influx - LabHS - power shellies.csv")
ffactory = os.path.join(fpath, "230420 influx - AFB.csv")

#### power

In [4]:
power = pd.read_csv(fpower,skiprows=3)
power = power.drop(columns=['Unnamed: 0','result'])

for c in ['_start','_stop','_time']: power[c] = pd.to_datetime(power[c])
power.info()

pd.DataFrame.to_csv(power, os.path.join(fpath, "power.csv"), index=None)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260909 entries, 0 to 260908
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   table         260909 non-null  int64              
 1   _start        260909 non-null  datetime64[ns, UTC]
 2   _stop         260909 non-null  datetime64[ns, UTC]
 3   _time         260909 non-null  datetime64[ns, UTC]
 4   _value        260909 non-null  float64            
 5   Device        260909 non-null  object             
 6   Location      260909 non-null  int64              
 7   Sensor        260909 non-null  object             
 8   _field        260909 non-null  object             
 9   _measurement  260909 non-null  object             
 10  host          260909 non-null  object             
 11  topic         260909 non-null  object             
dtypes: datetime64[ns, UTC](3), float64(1), int64(2), object(6)
memory usage: 23.9+ MB


In [5]:
power_time_min = power['_time'].min()
power_time_max = power['_time'].max()

power_time_range = power_time_max - power_time_min
power_time_range = (power_time_range / 1e9)

power_time_min ,power_time_max ,power_time_range.to_timedelta64()


(Timestamp('2023-04-04 16:09:25.378702087+0000', tz='UTC'),
 Timestamp('2023-04-20 09:57:50.383191985+0000', tz='UTC'),
 numpy.timedelta64(1360105,'ns'))

In [6]:
power['_time_hour'] = power['_time'].dt.hour.to_numpy()

power_per_hour = power.groupby('_time_hour')['_value'].agg(['sum']).reset_index()
print(power_per_hour.to_markdown())

|    |   _time_hour |              sum |
|---:|-------------:|-----------------:|
|  0 |            0 | 803179           |
|  1 |            1 | 832385           |
|  2 |            2 | 824132           |
|  3 |            3 | 820674           |
|  4 |            4 | 847309           |
|  5 |            5 | 825616           |
|  6 |            6 | 824371           |
|  7 |            7 | 817318           |
|  8 |            8 | 852501           |
|  9 |            9 | 847811           |
| 10 |           10 | 819008           |
| 11 |           11 | 847201           |
| 12 |           12 | 951188           |
| 13 |           13 | 899599           |
| 14 |           14 | 866023           |
| 15 |           15 | 860788           |
| 16 |           16 | 840408           |
| 17 |           17 | 850406           |
| 18 |           18 | 853921           |
| 19 |           19 | 842527           |
| 20 |           20 | 833393           |
| 21 |           21 | 833814           |
| 22 |          

In [40]:
power_per_hour_location = power.groupby(['_time_hour','Location'])['_value'].agg(['sum']).reset_index()

print(power_per_hour_location.sort_values(by=['Location','_time_hour']).to_markdown())

|    |   _time_hour |   Location |              sum |
|---:|-------------:|-----------:|-----------------:|
|  0 |            0 |       9306 |  65426.3         |
|  3 |            1 |       9306 |  68697.7         |
|  6 |            2 |       9306 |  68048.4         |
|  9 |            3 |       9306 |  72934.1         |
| 12 |            4 |       9306 |  61431           |
| 15 |            5 |       9306 |  61637.4         |
| 18 |            6 |       9306 |  62849.9         |
| 21 |            7 |       9306 |  71302.5         |
| 24 |            8 |       9306 |  72155           |
| 27 |            9 |       9306 |  64425.2         |
| 30 |           10 |       9306 |  61947           |
| 33 |           11 |       9306 |  59908.4         |
| 36 |           12 |       9306 |  71360           |
| 39 |           13 |       9306 |  64117           |
| 42 |           14 |       9306 |  64187.7         |
| 45 |           15 |       9306 |  61895.6         |
| 48 |           16 |       

In [48]:
print(f"devices: {power['Device'].unique()}")

devices: ['TraverseFlur' 'XRCool' 'XRL08' 'server1' 'server2' 'xair']


In [69]:
power_server = power.loc[power['Device'].isin(['server1', 'server2']),:]

print(f"server(1,2) power consumtion: {power_server._value.sum():.3f}")

server(1,2) power consumtion: 18352373.510


In [70]:
# power of grafic accelerator 

power_acc = power.loc[(power['Device']=='xair') & (power['_value'] > 0),:]

print(f"xair change {power_acc['_time'].min()}")

power_consumtion_before = power[power['_time']<power_acc['_time'].min()]._value.mean()
power_consumtion_after = power[power['_time']>=power_acc['_time'].min()]._value.mean()

print(f"power consumtion increase by accelerator from {power_consumtion_before:.3f} up to {power_consumtion_after:.3f}")

xair change 2023-04-05 07:50:39.828265379+00:00
power consumtion increase by accelerator from 77.863 up to 80.484


In [19]:
power_srv = power.loc[power['Device']=='server1',:].reindex()

outlier_value = 175
outlier_correction = 25

# correct outlier
power_srv.loc[power_srv['_value'] > outlier_value,'_value'] = power_srv.loc[power_srv['_value'] > outlier_value,'_value'].to_numpy() - outlier_correction

power_srv.loc[(power_srv['_value'] > outlier_value) 
              & ~(power_srv['_time'].dt.hour.isin([22,23,24,0,1,2,3]))
              ,:]


,table,_start,_stop,_time,_value,Device,Location,Sensor,_field,_measurement,host,topic
177634,3,2023-04-04 16:09:25.378000+00:00,2023-04-20 09:58:05.247037641+00:00,2023-04-13 11:58:46.866152481+00:00,185.44,server1,9308,power,value,shellies,XRMAINServer,shellies/9308/server1/relay/0/power


### factory

In [2]:
factory = pd.read_csv(ffactory,skiprows=3)
factory = factory.drop(columns=['Unnamed: 0','result'])

for c in ['_start','_stop','_time']:
    factory[c] = pd.to_datetime(factory[c])

factory.info()

pd.DataFrame.to_csv(factory, os.path.join(fpath, "factory.csv"), index=None)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31317 entries, 0 to 31316
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   table         31317 non-null  int64              
 1   _start        31317 non-null  datetime64[ns, UTC]
 2   _stop         31317 non-null  datetime64[ns, UTC]
 3   _time         31317 non-null  datetime64[ns, UTC]
 4   _value        31317 non-null  int64              
 5   Assembly      31317 non-null  object             
 6   Signal        31317 non-null  object             
 7   _field        31317 non-null  object             
 8   _measurement  31317 non-null  object             
 9   host          31317 non-null  object             
 10  topic         31317 non-null  object             
dtypes: datetime64[ns, UTC](3), int64(2), object(6)
memory usage: 2.6+ MB


In [3]:
# 

factory_time_range = factory['_time'].max() - factory['_time'].min()
print(f"time range {factory_time_range}")

factory_time_min = factory['_time'].min()
factory_time_max = factory['_time'].max()

factory_time_range = factory_time_max - factory_time_min
factory_time_range = (factory_time_range / 1e9)

factory_time_min ,factory_time_max ,factory_time_range.to_timedelta64()

time range 0 days 03:58:55.391193769


(Timestamp('2023-04-12 07:24:48.807770123+0000', tz='UTC'),
 Timestamp('2023-04-12 11:23:44.198963892+0000', tz='UTC'),
 numpy.timedelta64(14335,'ns'))

In [113]:
factory_assembly_signal = factory[['Assembly','Signal']].groupby('Assembly').agg({'Signal':pd.Series.nunique}).reset_index()
print(factory_assembly_signal.to_markdown())
print(f"signals: {len(factory['Signal'].unique())}")

|    | Assembly     |   Signal |
|---:|:-------------|---------:|
|  0 | Conveyor     |       54 |
|  1 | Distributing |       19 |
|  2 | Filling      |       37 |
|  3 | InOut        |        1 |
|  4 | Packaging    |       20 |
|  5 | Separating   |       14 |
|  6 | Unpacking    |       22 |
signals: 138


In [126]:
lories = factory.loc[(factory['Signal']=='KameraP') & (factory['_value']!=0),'_value'].unique()
print(f"{lories} count {len(lories)}")

[ 5 20 12 10] count 4


In [230]:
cycles = factory.loc[(factory['Signal']=='KameraP') & (factory['_value']!=0),['_time','_value']].sort_values(by=['_value','_time'])

cycles['dt'] = (cycles['_time'] - cycles['_time'].shift(1)).dt.total_seconds()
cycles.loc[(cycles['dt'] > 100) | (cycles['dt'] < 0),'dt'] = np.NaN

cycles = cycles[['_value','dt']].groupby(by=['_value'])['dt'].agg(['mean','std','count']).reset_index()
cycles.columns = ['lorry','_mean','_dev','_count']

cycles

,lorry,_mean,_dev,_count
0,5,61.775561,2.974102,13
1,10,61.823726,3.430666,11
2,12,62.279425,3.227177,11
3,20,61.774972,3.293820,13


In [7]:
frame = factory.loc[(factory['Signal'].isin(['KameraP','10B3'])) & (factory['Assembly'].isin(['Packaging','Conveyor'])),['_time','Signal','Assembly','_value']]
frame = frame[((frame['_value']!=0) & (frame['Signal']=='KameraP') & (frame['Assembly']=='Packaging')) | ((frame['Signal']=='10B3') & (frame['Assembly']=='Conveyor'))]
frame = frame.sort_values(by=['_time']).reset_index().drop(columns=['index'])

index = frame[(frame['Signal']=='KameraP') & (frame['Assembly']=='Packaging')].index

for k in list(zip(index[::1],index[1:-1:1])): 
    _id = frame.iloc[k[0],:]._value
    _value = frame.iloc[k[0]+1:k[1]-1,:]._value.sum()
    _time = frame.iloc[k[0],:]._time
    print(f"{_id:02d} |-{_value:1d}-| {_time.strftime('%H:%M:%S %d.%m.%Y')}")


05 |-4-| 07:28:48 12.04.2023
20 |-4-| 07:29:01 12.04.2023
12 |-4-| 07:29:13 12.04.2023
10 |-4-| 07:29:39 12.04.2023
05 |-4-| 07:29:51 12.04.2023
20 |-4-| 07:30:04 12.04.2023
12 |-4-| 07:30:16 12.04.2023
10 |-4-| 07:30:37 12.04.2023
05 |-4-| 07:30:49 12.04.2023
20 |-4-| 07:31:09 12.04.2023
12 |-4-| 07:31:21 12.04.2023
10 |-4-| 07:31:36 12.04.2023
05 |-4-| 07:31:48 12.04.2023
20 |-4-| 07:32:09 12.04.2023
12 |-4-| 07:32:21 12.04.2023
10 |-4-| 07:32:35 12.04.2023
05 |-4-| 07:32:47 12.04.2023
20 |-4-| 07:33:09 12.04.2023
12 |-4-| 07:33:21 12.04.2023
10 |-4-| 07:33:34 12.04.2023
05 |-4-| 07:33:46 12.04.2023
20 |-4-| 07:34:15 12.04.2023
12 |-4-| 07:34:27 12.04.2023
10 |-4-| 07:34:41 12.04.2023
05 |-4-| 07:34:52 12.04.2023
20 |-4-| 07:35:18 12.04.2023
12 |-4-| 07:35:30 12.04.2023
10 |-4-| 07:35:45 12.04.2023
05 |-4-| 07:35:57 12.04.2023
20 |-4-| 07:36:16 12.04.2023
12 |-8-| 07:36:28 12.04.2023
05 |-4-| 07:36:57 12.04.2023
20 |-8-| 07:37:17 12.04.2023
10 |-4-| 09:03:12 12.04.2023
05 |-4-| 09:03